In [1]:
# Import the required libraries
import numpy as np
import pandas as pd
import json
import requests
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import wget

In [2]:
CLIENT_ID = 'NY40LS44LI5LOLQLNG4JATGX55KPAO212YRFAJ1Z52MA3M1U'
CLIENT_SECRET = 'ERAVXBOIN4X54NKVHRCIMR02P1F033XT45XFJI5ZL3IFT0OF'
ACCESS_TOKEN = '5AXUAIRZSQSCYNYSZ44JC0PMBAJLBPGF1F1QZYE3QEXCBEO4'
VERSION = '20180604'

# Data required

## Import data of New York

In [3]:
# url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json'
# filename = wget.download(url)
# filename
# Like I already have the data only I need to call.

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [4]:
# Create our labels on the data
column_names =  ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
new_york = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    new_york = new_york.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
new_york.head()    

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [6]:
# Get the location of New York City
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location_NY = geolocator.geocode(address)
latitude_NY = location_NY.latitude
longitude_NY = location_NY.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude_NY, longitude_NY))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


## Import data of Toronto

In [7]:
# Like I already have the data only I need to call by pandas
df_1T = pd.read_excel ('D:\python\data analysis\Data_Toronto.xlsx')
df_1T.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
# Use the function drop to eliminate all values 'Not assigned' 
for i in range(0,df_1T.shape[0]):
    if df_1T.loc[i,'Borough']== 'Not assigned':
        df_1T=df_1T.drop(i)
        df_1T.reset_index(drop=True)
df_1T.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
# Download location Data https://cocl.us/Geospatial_data
df_location_1T = pd.read_csv('D:\python\data analysis\Geospatial_Coordinates.csv')
df_location_1T.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
# Merge our two datas into one on Postal Code like reference
toronto = pd.merge(df_1T, df_location_1T,how='left',on='Postal Code')
toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [11]:
# Get the location of Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location_T = geolocator.geocode(address)
latitude_T = location_T.latitude
longitude_T = location_T.longitude
print('The geograpical coordinate of Toronto {}, {}.'.format(latitude_T, longitude_T))

The geograpical coordinate of Toronto 43.6534817, -79.3839347.


## Data specifically of Brooklyn

In [12]:
# Get the data of Brooklyn
brooklyn_data = new_york[new_york['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [13]:
# Get the location of Brooklyn
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location_bro = geolocator.geocode(address)
latitude_bro = location_bro.latitude
longitude_bro = location_bro.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude_bro, longitude_bro))

The geograpical coordinate of Manhattan are 40.6501038, -73.9495823.


In [14]:
# Get a map of venues in Brooklyn

map_bro = folium.Map(location=[latitude_bro, longitude_bro], zoom_start=11)

for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bro)  
    
map_bro

In [15]:
# Get data of Gravesend
brooklyn_data.loc[4, 'Neighborhood']

'Gravesend'

In [16]:
# Get location of Gravesend
gravesend_latitude = brooklyn_data.loc[4, 'Latitude'] 
gravesend_longitud = brooklyn_data.loc[4, 'Longitude']

gravesend_name = brooklyn_data.loc[4, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(gravesend_name, 
                                                               gravesend_latitude, 
                                                               gravesend_longitud))

Latitude and longitude values of Gravesend are 40.59526001306593, -73.97347087708445.


In [17]:
# Get API
LIMIT = 100 
radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    gravesend_latitude, 
    gravesend_longitud, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=NY40LS44LI5LOLQLNG4JATGX55KPAO212YRFAJ1Z52MA3M1U&client_secret=ERAVXBOIN4X54NKVHRCIMR02P1F033XT45XFJI5ZL3IFT0OF&v=20180604&ll=40.59526001306593,-73.97347087708445&radius=500&limit=100'

In [18]:
results = requests.get(url).json()

In [19]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
# Get near venues to Brooklyn
venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues) 
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Joe's of Avenue U,Italian Restaurant,40.596971,-73.973677
1,Nuccios Bakery,Bakery,40.597097,-73.974483
2,Living Room Cafe,Lounge,40.596439,-73.977583
3,Dunkin',Donut Shop,40.596786,-73.976686
4,Blue Velvet Lounge,Bar,40.597162,-73.971599


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

25 venues were returned by Foursquare.


In [22]:
# Get data of nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
# Get neighbourhoods within North York
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude'])

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [24]:
# Show location of neighbourhoods
print(brooklyn_data.shape)
brooklyn_data.head()

(70, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [25]:
print('There are {} uniques categories.'.format(len(brooklyn_venues['Venue Category'].unique())))

There are 291 uniques categories.


In [26]:
# Show the numebr of shops in brooklyn
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]
brooklyn_onehot.head()

,Yoga Studio,Accessories Store,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,...,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yemeni Restaurant
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# Show the proportion, within brooklyn, of shops 
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,...,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yemeni Restaurant
0,Bath Beach,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.021277,0.021277,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.021277,0.0
1,Bay Ridge,0.011628,0.0,0.0,0.034884,0.0,0.0,0.0,0.0,0.0,...,0.011628,0.000000,0.011628,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
2,Bedford Stuyvesant,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.034483,0.034483,0.0,0.000000,0.0
3,Bensonhurst,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
4,Bergen Beach,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0


In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
# Show the 10 most common venues within each neighbourhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

brooklyn_venues_sorted = pd.DataFrame(columns=columns)
brooklyn_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    brooklyn_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

brooklyn_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Pharmacy,Chinese Restaurant,Bubble Tea Shop,Donut Shop,Sushi Restaurant,Fast Food Restaurant,Pizza Place,Dessert Shop,Italian Restaurant,Gas Station
1,Bay Ridge,Spa,Bagel Shop,Chinese Restaurant,Pizza Place,Italian Restaurant,American Restaurant,Greek Restaurant,Pharmacy,Bar,Grocery Store
2,Bedford Stuyvesant,Coffee Shop,Café,Pizza Place,Bar,Deli / Bodega,Bus Stop,New American Restaurant,Fried Chicken Joint,Gift Shop,Gourmet Shop
3,Bensonhurst,Chinese Restaurant,Pizza Place,Donut Shop,Italian Restaurant,Sushi Restaurant,Ice Cream Shop,Factory,Bar,Flower Shop,Halal Restaurant
4,Bergen Beach,Harbor / Marina,Baseball Field,Playground,Park,Athletics & Sports,Yoga Studio,Other Great Outdoors,Pakistani Restaurant,Outlet Store,Outdoors & Recreation
5,Boerum Hill,Coffee Shop,Dance Studio,Bar,French Restaurant,Sandwich Place,Bakery,Furniture / Home Store,Arts & Crafts Store,Cosmetics Shop,Grocery Store
6,Borough Park,Bank,Pizza Place,Fast Food Restaurant,Café,Deli / Bodega,Pharmacy,Bakery,Metro Station,Men's Store,Farmers Market
7,Brighton Beach,Eastern European Restaurant,Restaurant,Russian Restaurant,Beach,Pharmacy,Gourmet Shop,Sushi Restaurant,Mobile Phone Shop,Bank,Diner
8,Broadway Junction,Donut Shop,Fried Chicken Joint,Gas Station,Nightclub,Hotel,Grocery Store,Bus Station,Sandwich Place,Caribbean Restaurant,Tunnel
9,Brooklyn Heights,Deli / Bodega,Yoga Studio,Pizza Place,Cosmetics Shop,Coffee Shop,Pharmacy,Gym,Italian Restaurant,Park,Pet Store


## Data specifically of North York

In [30]:
# get the data of North York
NorthYork_data = toronto[toronto['Borough'] == 'North York'].reset_index(drop=True)
NorthYork_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [31]:
# Get the location of North York
address = 'North York'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [32]:
# Get a map of venues in North York
NorthYork_map = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(NorthYork_data['Latitude'], NorthYork_data['Longitude'], NorthYork_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(NorthYork_map)  
    
NorthYork_map

In [33]:
# Get a data of Glencairn
NorthYork_data.loc[4, 'Neighbourhood']

'Glencairn'

In [34]:
# Get the location of Glencairn
Glencairn_latitude = NorthYork_data.loc[4, 'Latitude'] 
Glencairn_longitude = NorthYork_data.loc[4, 'Longitude']

Glencairn_name = NorthYork_data.loc[4, 'Neighbourhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(Glencairn_name, 
                                                               Glencairn_latitude, 
                                                               Glencairn_longitude))

Latitude and longitude values of Glencairn are 43.709577, -79.4450726.


In [37]:
# Get API
LIMIT = 100 
radius = 500 
url_NoY = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Glencairn_latitude, 
    Glencairn_longitude, 
    radius, 
    LIMIT)
url_NoY

'https://api.foursquare.com/v2/venues/explore?&client_id=NY40LS44LI5LOLQLNG4JATGX55KPAO212YRFAJ1Z52MA3M1U&client_secret=ERAVXBOIN4X54NKVHRCIMR02P1F033XT45XFJI5ZL3IFT0OF&v=20180604&ll=43.709577,-79.4450726&radius=500&limit=100'

In [38]:
results_NoY = requests.get(url_NoY).json()

In [39]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [40]:
# Get near venues to North York
venues = results_NoY['response']['groups'][0]['items']
nearby_venues_NoY = pd.json_normalize(venues) 
filtered_columns_NoY = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_NoY =nearby_venues_NoY.loc[:, filtered_columns_NoY]
nearby_venues_NoY['venue.categories'] = nearby_venues_NoY.apply(get_category_type, axis=1)
nearby_venues_NoY.columns = [col.split(".")[-1] for col in nearby_venues_NoY.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Joe's of Avenue U,Italian Restaurant,40.596971,-73.973677
1,Nuccios Bakery,Bakery,40.597097,-73.974483
2,Living Room Cafe,Lounge,40.596439,-73.977583
3,Dunkin',Donut Shop,40.596786,-73.976686
4,Blue Velvet Lounge,Bar,40.597162,-73.971599


In [41]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

25 venues were returned by Foursquare.


In [42]:
# Get data of nearby venues 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [43]:
#Get the neighbourhoods in North York
NorthYork_venues = getNearbyVenues(names=NorthYork_data['Neighbourhood'],
                                   latitudes=NorthYork_data['Latitude'],
                                   longitudes=NorthYork_data['Longitude'])

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [45]:
# Get location and category of venues
print(NorthYork_venues.shape)
NorthYork_venues.head()

(240, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [46]:
# Get the total of venues in Brooklyn by neighbourhood
NorthYork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Don Mills,25,25,25,25,25,25
Downsview,16,16,16,16,16,16
"Fairview, Henry Farm, Oriole",64,64,64,64,64,64
Glencairn,4,4,4,4,4,4
Hillcrest Village,6,6,6,6,6,6
Humber Summit,3,3,3,3,3,3


In [47]:
print('There are {} uniques categories.'.format(len(NorthYork_venues['Venue Category'].unique())))

There are 100 uniques categories.


In [48]:
# Show the number of venues in Brooklyn
NorthYork_onehot = pd.get_dummies(NorthYork_venues[['Venue Category']], prefix="", prefix_sep="")

NorthYork_onehot['Neighborhood'] = NorthYork_venues['Neighborhood'] 

fixed_columns = [NorthYork_onehot.columns[-1]] + list(NorthYork_onehot.columns[:-1])
NorthYork_onehot = NorthYork_onehot[fixed_columns]

NorthYork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Sporting Goods Shop,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
# Show the proportion, within North York, of venues
NorthYork_grouped = NorthYork_onehot.groupby('Neighborhood').mean().reset_index()
NorthYork_grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Sporting Goods Shop,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0000,0.00,0.00,0.0,0.00,0.0000,0.0,0.090909,...,0.00,0.045455,0.045455,0.0,0.00,0.0,0.0,0.0,0.0,0.0
1,Bayview Village,0.0,0.0000,0.00,0.00,0.0,0.00,0.0000,0.0,0.250000,...,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,"Bedford Park, Lawrence Manor East",0.0,0.0000,0.04,0.00,0.0,0.00,0.0000,0.0,0.000000,...,0.00,0.000000,0.040000,0.0,0.04,0.0,0.0,0.0,0.0,0.0
3,Don Mills,0.0,0.0000,0.00,0.04,0.0,0.04,0.0000,0.0,0.000000,...,0.04,0.040000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0
4,Downsview,0.0,0.0625,0.00,0.00,0.0,0.00,0.0625,0.0,0.062500,...,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0


In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [51]:
# Show the 10 most common venues within each neighbourhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = NorthYork_grouped['Neighborhood']

for ind in np.arange(NorthYork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NorthYork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pet Store,Shopping Mall,Mobile Phone Shop,Park,Pharmacy,Pizza Place,Diner,Restaurant
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Liquor Store,Park,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Greek Restaurant,Grocery Store,Indian Restaurant,Juice Bar,Liquor Store,Fast Food Restaurant,Pharmacy
3,Don Mills,Gym,Coffee Shop,Clothing Store,Restaurant,Bike Shop,Discount Store,Construction & Landscaping,Chinese Restaurant,Caribbean Restaurant,Café
4,Downsview,Grocery Store,Park,Gym / Fitness Center,Bank,Korean Restaurant,Shopping Mall,Food Truck,Business Service,Baseball Field,Hotel
5,"Fairview, Henry Farm, Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Cosmetics Shop,Restaurant,Juice Bar,Chocolate Shop,Bakery,Bank
6,Glencairn,Smoke Shop,Convenience Store,Bakery,Japanese Restaurant,Juice Bar,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Men's Store,Mediterranean Restaurant
7,Hillcrest Village,Golf Course,Athletics & Sports,Pool,Mediterranean Restaurant,Fast Food Restaurant,Dog Run,Accessories Store,Korean Restaurant,Mobile Phone Shop,Miscellaneous Shop
8,Humber Summit,Pizza Place,Furniture / Home Store,Intersection,Accessories Store,Korean Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Men's Store,Mediterranean Restaurant
9,"Humberlea, Emery",Baseball Field,Accessories Store,Juice Bar,Movie Theater,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Men's Store,Mediterranean Restaurant,Massage Studio


In [52]:
# let's see the table of brooklyn to note the difference
brooklyn_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Pharmacy,Chinese Restaurant,Bubble Tea Shop,Donut Shop,Sushi Restaurant,Fast Food Restaurant,Pizza Place,Dessert Shop,Italian Restaurant,Gas Station
1,Bay Ridge,Spa,Bagel Shop,Chinese Restaurant,Pizza Place,Italian Restaurant,American Restaurant,Greek Restaurant,Pharmacy,Bar,Grocery Store
2,Bedford Stuyvesant,Coffee Shop,Café,Pizza Place,Bar,Deli / Bodega,Bus Stop,New American Restaurant,Fried Chicken Joint,Gift Shop,Gourmet Shop
3,Bensonhurst,Chinese Restaurant,Pizza Place,Donut Shop,Italian Restaurant,Sushi Restaurant,Ice Cream Shop,Factory,Bar,Flower Shop,Halal Restaurant
4,Bergen Beach,Harbor / Marina,Baseball Field,Playground,Park,Athletics & Sports,Yoga Studio,Other Great Outdoors,Pakistani Restaurant,Outlet Store,Outdoors & Recreation
5,Boerum Hill,Coffee Shop,Dance Studio,Bar,French Restaurant,Sandwich Place,Bakery,Furniture / Home Store,Arts & Crafts Store,Cosmetics Shop,Grocery Store
6,Borough Park,Bank,Pizza Place,Fast Food Restaurant,Café,Deli / Bodega,Pharmacy,Bakery,Metro Station,Men's Store,Farmers Market
7,Brighton Beach,Eastern European Restaurant,Restaurant,Russian Restaurant,Beach,Pharmacy,Gourmet Shop,Sushi Restaurant,Mobile Phone Shop,Bank,Diner
8,Broadway Junction,Donut Shop,Fried Chicken Joint,Gas Station,Nightclub,Hotel,Grocery Store,Bus Station,Sandwich Place,Caribbean Restaurant,Tunnel
9,Brooklyn Heights,Deli / Bodega,Yoga Studio,Pizza Place,Cosmetics Shop,Coffee Shop,Pharmacy,Gym,Italian Restaurant,Park,Pet Store


##### We can note that within Brooklyn there are some beaches; however, wihtin North York there is not any. This is a grographic factor and if you look at a map on internet you will see that Brooklyn has a limit with the sea while North York does not. With this in mind, I can say that North York has more space(also remember that North York has larger neoghbourhoods) whereas Brooklyn has more kind of ways de start a business, because it has a great beach. Also it is easy to note that in Brooklyn people prefer leisure. So you have a town where people love leisure, therefore if you wamt to star a business in one of these mentioned cities, a correct desicion would be invest in entertaiment in Brooklyn.